In [ ]:
import random
import re
import os
import sys
import tqdm
import xml.etree.ElementTree as ET

import numpy as np
from nltk import tokenize

import string

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import Sequence


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
class Debug:
    def __init__(self, debug_mode=True):
        self.debug_mode = debug_mode
        self.flag = {}

    def log(self, target, flag=None):
        if self.debug_mode:
            if flag is None:
                print(target)
            else:
                if flag in self.flag.keys():
                    if self.flag[flag]:
                        print(target)

    def set_flag(self, flag: str, val: bool):
        self.flag[flag] = val

debug = Debug(True)

In [ ]:
class GeneratorExceptions(Exception):
    """
    The Exception class for tracking all exceptions raised in data generator
    Param
        text: the displayed text
    """
    def __init__(self, text: str):
        self.text = text

class data_generator(Sequence):
    def __init__(self, 
                 dataset_file_path : str="data/dataset/nysk.xml", 
                 processed_dataset_path: str ="data/processed_dataset/",
                 batch_size = 2,
                 shuffle = True):
        self.dataset_file_path = dataset_file_path
        self.processed_dataset_path = processed_dataset_path
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.file_list = []
        self.encoder_input_data = []
        self.decoder_input_data = []
        self.decoder_target_data = []
        self.char_list = []
        self.characters_set = set()
        self.tokens_count = 0
        self.max_sequence_len = 0
        self.token_index = None
        self.input_texts = None
        self.model = None
        
        self.epoch_count = 0
        
        self.validation_split=0.1
    
    def generate_char_dict(self):
        
        for i in string.ascii_letters:
            self.characters_set.add(i)
        
        for i in "1234567890\n\t.,!(){}\"\' ":
            self.characters_set.add(i)
        
        self.char_list = sorted(list(self.characters_set))
        self.tokens_count = len(self.char_list)
        self.token_index = dict([(char, i) for i, char in enumerate(self.characters_set)])

        
    def preprocess_data(self, override=False, max:int=3):
        self.file_list = []
        if os.path.isfile(self.dataset_file_path):
            if not os.path.isdir(self.processed_dataset_path):
                os.mkdir(self.processed_dataset_path)

            with open(self.dataset_file_path, "r", encoding="utf-8") as f:
                doc = ET.ElementTree(file=f)

            root = doc.getroot()
            
            pos = 0
            
            for item in tqdm.tqdm(root):
                if max != -1:
                    if pos > max:
                        break
                    pos += 1
                news_id = item.findtext('docid')
                source = item.findtext('source')
                url = item.findtext('url')
                title = item.findtext('title')
                summary = item.findtext('summary')
                raw_text = item.findtext('text')

                title = re.sub(r"<.*>", "", title)
                title = re.sub(r"\W", "_", title)
                title = f"{news_id}_{title[:10]}"
                
                res = tokenize.sent_tokenize(raw_text)
                sentences_count = len(res)
                       
                text = ""
                
                
                for s in res:
                    
                    if len(s) > 200:
                        continue
                    else:
                        if len(s) > self.max_sequence_len:
                            self.max_sequence_len = len(s)
                        t_sentence = ""
                        for c in s:
                            if c in self.characters_set:
                                t_sentence += c
                        t_sentence += "\t"
                        text += t_sentence
                
                fp = f"{self.processed_dataset_path}{title}_{sentences_count}.txt"
                
                if not os.path.isfile(fp) or override:
                    with open(fp, 'w', encoding='utf-8') as f:
                        f.write(text)
            self.max_sequence_len += 3
            
        else:
            raise GeneratorExceptions("Path doesn't exist")
    
    def generate_file_list(self, length:int=-1):
        temp = os.listdir(self.processed_dataset_path)
        self.file_list = []
        t_list = []
        for i in temp:
            t_list.append(f"{self.processed_dataset_path}{i}")
        self.file_list = t_list[1:length]
    
    def process_data(self, text):
        input_texts = []
        target_texts = []
        
        for i in range(0, len(text)-1):
            input_t = f"\t{text[i]}\n"
            target_t = f"\t{text[i+1]}\n"
            input_texts.append(input_t)
            target_texts.append(target_t)
        
        temp_encoder_input_data = np.zeros(
            (len(input_texts), self.max_sequence_len, self.tokens_count), dtype="float32"
        )
        temp_decoder_input_data = np.zeros(
            (len(input_texts), self.max_sequence_len, self.tokens_count), dtype="float32"
        )
        temp_decoder_target_data = np.zeros(
            (len(input_texts), self.max_sequence_len, self.tokens_count), dtype="float32"
        )

        for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, char in enumerate(input_text):
                temp_encoder_input_data[i, t, self.token_index[char]] = 1.0
                temp_encoder_input_data[i, t + 1 :, self.token_index[" "]] = 1.0
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                temp_decoder_input_data[i, t, self.token_index[char]] = 1.0
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    temp_decoder_target_data[i, t - 1, self.token_index[char]] = 1.0
                    temp_decoder_input_data[i, t + 1 :, self.token_index[" "]] = 1.0
                    temp_decoder_target_data[i, t:, self.token_index[" "]] = 1.0
        
        self.encoder_input_data = temp_encoder_input_data
        self.decoder_input_data = temp_decoder_input_data
        self.decoder_target_data = temp_decoder_target_data
        
        self.input_texts = input_texts
        
        return
    
    def __len__(self):
        return len(self.file_list) // self.batch_size
    
    def __getitem__(self, index):
        files = self.file_list[index * self.batch_size:(index + 1) * self.batch_size]
        res = []
        for fl in files:
            with open(fl, 'r', encoding='utf-8') as dt:
                text = dt.read()
                temp_list = text.split("\t")
                res.extend(temp_list)
        
        self.process_data(res)
        return [self.encoder_input_data, self.decoder_input_data], self.decoder_target_data,
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.file_list)
        model.save_weights(f"ckpt/ckpt-{self.epoch_count}.hdf5")
        self.epoch_count += 1
        return
    

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# Testing data generator
DataGenerator = data_generator()
DataGenerator.generate_char_dict()
print(DataGenerator.token_index)

In [ ]:
DataGenerator.preprocess_data(override=True, max=2000)

In [ ]:
print(DataGenerator.tokens_count)
print(DataGenerator.max_sequence_len)

In [ ]:
DataGenerator.generate_file_list()
print(len(DataGenerator))
print(len(DataGenerator.file_list))
# print(temp_dg.__getitem__(1))

In [ ]:
def generate_model(num_encoder_tokens, num_decoder_tokens, latent_dim=256):
      
    encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
    encoder = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs_, state_h, state_c = encoder(encoder_inputs)
    
    encoder_states = [state_h, state_c]
    
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
    
    decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    model.summary()
    
    return model

In [ ]:
latent_dim = 256

model = generate_model(num_encoder_tokens=DataGenerator.tokens_count, 
                       num_decoder_tokens=DataGenerator.tokens_count,
                       latent_dim=latent_dim)

model_name = "Model\SeqToSeq_Model"

model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
print(sys.getsizeof(model))
# model.load_weights("ckpt/ckpt-0005.ckpt")
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Don't execute this one for now
import nltk
nltk.download('punkt')

epochs = 10

print(DataGenerator.token_index)
DataGenerator.model = model

checkpoint_path = "ckpt/ckpt-{epoch:04d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=DataGenerator.batch_size)

model.fit(DataGenerator, 
          epochs=epochs, 
          verbose=1)

In [ ]:
model.save_weights("Model/test.weights.hdf5")
# keras.models.save_model(model, model_name)

In [ ]:
def decode_sequence(DataGenerator: data_generator, input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, DataGenerator.tokens_count))
    target_seq[0, 0, DataGenerator.token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        print(output_tokens)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > DataGenerator.max_sequence_len:
            stop_condition = True

        target_seq = np.zeros((1, 1, DataGenerator.tokens_count))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [ ]:
new_model = generate_model(num_encoder_tokens=DataGenerator.tokens_count, 
                           num_decoder_tokens=DataGenerator.tokens_count,
                           latent_dim=latent_dim)
new_model.compile(
    optimizer="rmsprop", 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)

new_model.summary()

new_model.load_weights("Model/test.weights.hdf5")
# new_model.load_weights("ckpt/ckpt-0005.ckpt")

In [ ]:
encoder_inputs = new_model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = new_model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = new_model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_5")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_6")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = new_model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = new_model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in DataGenerator.token_index.items())
reverse_target_char_index = dict((i, char) for char, i in DataGenerator.token_index.items())

In [ ]:
for seq_index in range(1):
    print(seq_index)
    DataGenerator.__getitem__(0)
    input_seq = DataGenerator.encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(DataGenerator, input_seq)
    print("-")
    print("Input sentence:", DataGenerator.input_texts[seq_index])
    for i in decoded_sentence:
        print(f"{i}, {ord(i)}")
    print("Decoded sentence:", decoded_sentence)
